In [4]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.metrics import roc_curve, auc

In [8]:
SCORES_DIR = "./eval/scores"
ECAPA_DIR = SCORES_DIR + "/ecapa"
ECAPA_SCHED_DIR = SCORES_DIR + "/ecapa-scheduler"

In [9]:
def load_scores(file_path):
    print(f"Loading scores from {file_path}")
    pd.read_csv(file_path, sep=' ', names=['label', 'score'])
    
def calculate_eer(y, y_score):
    fpr, tpr, thresholds = roc_curve(y, y_score)
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    return eer

def calculate_minDCF(y, y_scores, p_target=0.01, c_miss=1, c_fa=1):
    """Calculate the minimum Detection Cost Function (minDCF)."""
    fpr, tpr, thresholds = roc_curve(y, y_scores)
    fnr = 1 - tpr
    dcf = p_target * c_miss * fnr + (1 - p_target) * c_fa * fpr
    min_dcf = min(dcf)
    return min_dcf

In [18]:
def plot_metrics(epochs, metrics, title, ylabel):
    plt.figure(figsize=(10, 5))
    sns.lineplot(x=epochs, y=metrics)
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()
   
def process_model_scores(model, model_path, epochs):
    eers = []
    mindcfs = []
    for epoch in epochs:
        file_path = f"{model_path}/{model}.{epoch}.scores"
        df = load_scores(file_path)
        eer = calculate_eer(df['label'], df['score'])
        min_dcf = calculate_minDCF(df['label'], df['score'])
        eers.append(eer)
        mindcfs.append(min_dcf)
    return eers, mindcfs

In [19]:
epochs = [15, 20, 25, 30, 35]
ecapa_eers, ecapa_mindcfs = process_model_scores('ecapa', ECAPA_DIR, epochs)

TypeError: 'NoneType' object is not subscriptable